## Run blitzGSEA

In [1]:
import os
import pandas as pd
import blitzgsea as blitz

In [2]:
def load_custom_gmt(path):
    """
    Parse a GMT file into a dict: {term_name: [gene1, gene2, ...], …}
    """
    with open(path, 'r') as f:
        return {
            parts[0]: parts[2:]   # skip description at index 1
            for line in f
            if (parts := line.strip().split('\t')) and len(parts) > 2
        }


def run_gsea_pandas(input_tsv, gmt_file, output_tsv=None, processes=4):
    """
    Reads TSV, renames columns, runs GSEA with custom pathways, saves as TSV.
    
    Parameters
    ----------
    input_tsv : str
        Input file with at least 'symbol' and 'globalScore' columns.
    gmt_file : str
        Path to custom GMT file.
    processes : int
        Number of processes for GSEA.
    output_tsv : str or None
        Custom output filename. If None, defaults to <input_basename>_gsea.tsv
    """
    # Load library
    library_sets = load_custom_gmt(gmt_file)

    # Read input TSV
    df = pd.read_csv(input_tsv, sep="\t", header=0, index_col=None)
    
    # Create the expected format for blitz.gsea: columns [0, 1]
    gsea_df = pd.DataFrame()
    gsea_df[1] = df['symbol']  # gene symbols (matching GMT library)
    gsea_df[0] = pd.to_numeric(df['globalScore'], errors='coerce')  # scores
    
    # Drop rows with NaN scores
    gsea_df = gsea_df.dropna(subset=[0])
    
    print(f"GSEA input shape: {gsea_df.shape}")
    print(f"GSEA input columns: {gsea_df.columns.tolist()}")
    print(f"GSEA input sample:\n{gsea_df.head()}")
    
    # Run GSEA
    res_df = blitz.gsea(gsea_df, library_sets, processes=processes).reset_index(names="Term")

    # Add propagated_edge after GSEA computation is complete
    res_df["propagated_edge"] = res_df["Term"].apply(
        lambda t: ",".join(library_sets.get(t, [])) if library_sets.get(t) else ""
    )

    # Extract ID and clean Term
    term_series = res_df["Term"]
    res_df["ID"] = term_series.str.extract(r"\[([^\]]+)\]", expand=False).fillna("")
    res_df["Term"] = term_series.str.replace(r"\s*\[[^\]]+\]", "", regex=True).str.strip()

    # Ensure leading_edge is a string
    if "leading_edge" in res_df.columns:
        res_df["leading_edge"] = res_df["leading_edge"].apply(
            lambda x: ",".join(map(str, x)) if isinstance(x, (list, tuple)) else str(x)
        )

    # Reorder columns
    first_cols = ["Term", "ID"]
    res_df = res_df[first_cols + [c for c in res_df.columns if c not in first_cols]]

    # Determine output filename
    if output_tsv is None:
        output_tsv = f"{os.path.splitext(input_tsv)[0]}_gsea.tsv"

    # Save output
    res_df.to_csv(output_tsv, sep="\t", index=False)

    print(f"GSEA results saved to {output_tsv}")
    return res_df

### Reactome from database

In [ ]:
scores = "/Users/polina/genetics_gsea/data/input/geneset_disease_zscore.tsv"
library = "/Users/polina/targets-from-pathways/gsea/Reactome_2025/ReactomePathways_merged.gmt.gmt"
output_name = "/Users/polina/genetics_gsea/data/gsea/from_database/disease_zscore_reactome_2025.tsv"
output_name = "/Users/polina/genetics_gsea/data/gsea/from_database/disease_zscore_reactome_2025.tsv"

run_gsea_pandas(scores, library, processes=4, output_tsv=output_name)

GSEA input shape: (8285, 2)
GSEA input columns: [1, 0]
GSEA input sample:
        1         0
0  CDKN2B  8.694865
1     ABO  8.224087
2     FTO  7.753310
3   SH2B3  6.811755
4    APOE  6.340978
